
Demos - Geo - GeoPandas - NYC taxi trips
========================================


# References
* [Notebook - Geo - Demos - GeoPandas - NYC taxi trips (this notebook)](https://github.com/data-engineering-helpers/databricks-example/blob/main/ipython-notebooks/demos-geo-geopandas-nyc-taxi-trips.ipynb)
* See [Notebook - Geo - Demos - prep-nyc-taxi-geospatial-data](https://github.com/data-engineering-helpers/databricks-example/blob/main/ipython-notebooks/demos-geo-prep-nyc-taxi-geospatial-data.ipynb)
  for details on how to retrieve the data files, and why this notebook cannot work as is
  (mainly because the geo-coordinates are no longer available in the original data files from the
  NYC autorithies)

## GeoPandas
* GeoPandas home page: http://geopandas.org/

## Data
* https://www.databricks.com/notebooks/prep-nyc-taxi-geospatial-data.html
  + => [Notebook - Geo - Demos - prep-nyc-taxi-geospatial-data](https://github.com/data-engineering-helpers/databricks-example/blob/main/ipython-notebooks/demos-geo-prep-nyc-taxi-geospatial-data.ipynb) (forked notebook to adapt with new format of data)

## GeoPandas
__Option-1: Using [DBUtils Library Import](https://docs.databricks.com/dev-tools/databricks-utils.html#library-utilities) within Notebook (see cell #2).__

__Option-2: Using [Databricks ML Runtime](https://docs.databricks.com/runtime/mlruntime.html#mlruntime) which includes Anaconda (not used).__

* [Install Cluster Libraries](https://docs.databricks.com/libraries.html#install-a-library-on-a-cluster):
 * geopandas PyPI Coordinates: `geopandas`
 * shapely PyPI Coordinates: `shapely`

In [0]:
%fs ls /tmp/nyc-taxi/taxi-zones/csv/

path,name,size,modificationTime
dbfs:/tmp/nyc-taxi/taxi-zones/csv/taxi_zone_lookup.csv,taxi_zone_lookup.csv,12322,1699131546000
dbfs:/tmp/nyc-taxi/taxi-zones/csv/taxi_zones.csv,taxi_zones.csv,3755058,1699132695000


In [0]:
%pip install geopandas

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, shape
from shapely import wkb, wkt
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType,DecimalType
from pyspark.sql.functions import pandas_udf, PandasUDFType
import shapely.speedups
#shapely.speedups.enable() # this makes some spatial queries run faster

In [0]:
df_csv = pd.read_csv("/dbfs/tmp/nyc-taxi/taxi-zones/csv/taxi_zones.csv")
df_csv['the_geom'] = df_csv['the_geom'].apply(wkt.loads)
gdf  = gpd.GeoDataFrame(df_csv, geometry='the_geom')
sc.broadcast(gdf)
def find_borough(latitude, longitude): 
    mgdf = gdf.apply(lambda x: x['borough'] if x['the_geom'].intersects(Point(longitude,latitude)) else None, axis=1)
    idx = mgdf.first_valid_index()
    first_valid_value = mgdf.loc[idx] if idx is not None else None
    return first_valid_value
find_borough_udf = udf(find_borough, StringType())

In [0]:
# test the function
find_borough( 40.69943618774414,-73.9920883178711)

'Brooklyn'

In [0]:
df_raw = spark.read.parquet("/tmp/nyc-taxi/parquet/green_tripdata_*.parquet")
#df_raw = spark.read.format("delta").load("/ml/blogs/geospatial/delta/nyc-green")
df_raw_borough = df_raw.sample(False, 0.01).withColumn("pickup_borough", find_borough_udf(col("pickup_latitude"),col("pickup_longitude")))

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1373382088444214>, line 3
      1 df_raw = spark.read.parquet("/tmp/nyc-taxi/parquet/green_tripdata_*.parquet")
      2 #df_raw = spark.read.format("delta").load("/ml/blogs/geospatial/delta/nyc-green")
----> 3 df_raw_borough = df_raw.sample(False, 0.01).withColumn("pickup_borough", find_borough_udf(col("pickup_latitude"),col("pickup_longitude")))

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:5033, in DataFrame.withColumn(self, colName, col)
   502

In [0]:
display(df_raw_borough.select(["pickup_borough","pickup_datetime","pickup_latitude","pickup_longitude"]))

pickup_borough,pickup_datetime,pickup_latitude,pickup_longitude
Brooklyn,2014-04-10 21:39:08,40.68616485595703,-73.99085235595703
Manhattan,2014-04-10 21:40:00,40.8574104309082,-73.93215942382812
Brooklyn,2014-04-10 21:40:19,40.681095123291016,-73.95547485351562
Brooklyn,2014-04-10 21:46:54,40.679954528808594,-73.94032287597656
Queens,2014-04-10 21:48:25,40.72090530395508,-73.84422302246094
Brooklyn,2014-04-10 21:49:10,40.70438003540039,-73.98664093017578
Brooklyn,2014-04-10 21:49:43,40.68617248535156,-73.97416687011719
Queens,2014-04-10 21:50:50,40.75352096557617,-73.91926574707031
Queens,2014-04-10 21:53:11,40.72126007080078,-73.84423828125
Brooklyn,2014-04-10 21:53:21,40.654842376708984,-73.92778015136719
